# Name -- ADVAIT GURUNATH CHAVAN
# Email -- advaitchavan135@gmail.com

## Task -- 2

### You need to create a prototype pricing model that can go through further validation and testing before being put into production. Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client. 

### You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

### The input parameters that should be taken into account for pricing are:

#### Injection dates. 
#### Withdrawal dates.
#### The prices at which the commodity can be purchased/sold on those dates.
#### The rate at which the gas can be injected/withdrawn.
#### The maximum volume that can be stored.
#### Storage costs.
#### Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs

In [9]:
import pandas as pd
import numpy as np

In [3]:
# Reading the dataset
data = pd.read_csv('Nat_Gas.csv')

In [4]:
data.head()

,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


In [28]:
data.tail()

,Dates,Prices
43,5/31/24,11.4
44,6/30/24,11.5
45,7/31/24,11.6
46,8/31/24,11.5
47,9/30/24,11.8


In [15]:
import pandas as pd

def price_storage_contract(
    injection_dates: list[str],
    withdrawal_dates: list[str],
    daily_price: pd.Series,
    inject_rate: float,
    withdraw_rate: float,
    max_volume: float,
    storage_cost: float
) -> float:
    """
    Price a simple gas‐storage contract.

    Parameters
    ----------
    injection_dates : list[str]
        Dates ("YYYY-MM-DD") when gas is injected.
    withdrawal_dates : list[str]
        Dates ("YYYY-MM-DD") when gas is withdrawn.
    daily_price : pd.Series
        Series of daily gas prices, indexed by pd.DatetimeIndex.
    inject_rate : float
        Volume injected at each injection date.
    withdraw_rate : float
        Volume withdrawn at each withdrawal date.
    max_volume : float
        Maximum storage capacity.
    storage_cost : float
        Daily cost per unit volume stored.

    Returns
    -------
    float
        Net contract value (positive means profit).
    """

    # 1. Build event list
    events = []
    for d in injection_dates:
        events.append({
            "date": pd.to_datetime(d),
            "dvol":  inject_rate   # positive volume change
        })
    for d in withdrawal_dates:
        events.append({
            "date": pd.to_datetime(d),
            "dvol": -withdraw_rate  # negative volume change
        })

    # 2. Sort by date
    events.sort(key=lambda x: x["date"])

    # 3. Iterate events, track storage level, cash flows, storage cost
    storage_level = 0.0
    net_value = 0.0
    prev_date = events[0]["date"]

    for ev in events:
        curr_date = ev["date"]
        # 3.1 Storage cost accrual since last event
        days = (curr_date - prev_date).days
        if days > 0:
            net_value -= storage_level * storage_cost * days

        # 3.2 Volume change and feasibility checks
        new_level = storage_level + ev["dvol"]
        if new_level < 0:
            raise ValueError(f"Attempt to withdraw more than stored on {curr_date.date()}")
        if new_level > max_volume:
            raise ValueError(f"Storage capacity exceeded on {curr_date.date()}")
        
        # 3.3 Cash flow of injection/withdrawal
        try:
            price = float(daily_price.loc[curr_date])
        except KeyError:
            raise KeyError(f"No price available for {curr_date.date()}")
        
        # injection (dvol>0): cost → negative; withdrawal (dvol<0): revenue → positive
        net_value -= price * ev["dvol"]
        
        # Update state
        storage_level = new_level
        prev_date = curr_date

    # 4. Optional: Mark‐to‐market any remaining inventory at last known price
    if storage_level > 0:
        last_price = float(daily_price.loc[prev_date])
        net_value += storage_level * last_price

    return net_value

In [29]:
if __name__ == "__main__":
    # 1) Build a toy price series (daily price = 2.0 + sin curve)
    idx = pd.date_range("2020-10-31", "2024-09-30", freq="D")
    price_series = pd.Series(2.0 + 0.5 * np.sin(2*np.pi*idx.dayofyear/365), index=idx)

    # 2) Define a simple inject‐then‐withdraw schedule
    injections  = ["2022-03-01", "2022-04-01", "2023-05-01"]
    withdrawals = ["2022-10-01", "2022-11-01", "2023-12-01"]

    value = price_storage_contract(
        injection_dates  = injections,
        withdrawal_dates = withdrawals,
        daily_price      = price_series,
        inject_rate      = 100.0,
        withdraw_rate    = 100.0,
        max_volume       = 500.0,
        storage_cost     = 0.045
    )

    print(f"Net contract value: {value:.2f}")


Net contract value: -3143.15


In [34]:
if __name__ == "__main__":
    # 1) Build a toy price series (daily price = 2.0 + sin curve)
    idx = pd.date_range("2022-01-01", "2023-12-31", freq="D")
    price_series = pd.Series(2.0 + 0.5 * np.sin(2*np.pi*idx.dayofyear/365), index=idx)

    # 2) Define a simple inject‐then‐withdraw schedule
    injections  = ["2022-03-01", "2022-03-31", "2023-05-01"]
    withdrawals = ["2022-10-01", "2023-12-01", "2023-12-01"]

    value = price_storage_contract(
        injection_dates  = injections,
        withdrawal_dates = withdrawals,
        daily_price      = price_series,
        inject_rate      = 100.0,
        withdraw_rate    = 100.0,
        max_volume       = 500.0,
        storage_cost     = 0.0035
    )

    print(f"Net contract value: {value:.2f}")


Net contract value: -599.19
